# Time Domain Science Cases in the Galactic Plane

The purpose of this notebook is to evaluate the science cases that require time domain observations.  Here I distinguish these cases from those requesting proper motion measurements, since they tend to be distinct in their observational requirements.  For the purposes of this analysis, time domain science is identifed as those cases which requested more than 3 visits per field pointing over the lifetime of the survey.  

In [1]:
from os import path, getcwd
from sys import path as pythonpath
pythonpath.append(path.join(getcwd(), '..'))
import config_utils
import survey_footprints
import regions
import healpy as hp
from mw_plot import MWSkyMap, MWSkyMapBokeh
from astropy_healpix import HEALPix
from astropy import units as u
from astropy.coordinates import Galactic, TETE, SkyCoord, ICRS
from astropy.table import Table, Column
import numpy as np
from sklearn.cluster import KMeans
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
from os import path
%matplotlib inline


# Configure path to local repository
root_dir = '/Users/rstreet/software/rgps'

# HEALpixel grid resolution
NSIDE = 64
PIXAREA = hp.nside2pixarea(NSIDE, degrees=True)

First we load the simulation parameters for reference.

In [2]:
sim_config = config_utils.read_config(path.join(getcwd(), '..', 'config', 'sim_config.json'))

The requirements of all science cases proposed as White Papers and Science Pitches are described in the configuration file for this package:

In [3]:
science_cases = config_utils.read_config(path.join(root_dir, 'config', 'rgps_science_cases.json'))

Science cases requesting time domain observations are identifed in the configurations with a Boolean 'time_domain' key.  
So we can extract the information for just those cases for further analysis. 

In [4]:
time_domain_science = {author: info for author, info in science_cases.items() if info['time_domain']}
authors_list = [x for x in time_domain_science.keys()]
authors_list.sort()
print('Number of time domain science cases = ' + str(len(time_domain_science)))
print('\n')
print('Time domain authors and visit intervals [hrs] requested per region and filter:')
for author, params in time_domain_science.items():
    print(author + ': ')
    for optic in sim_config['OPTICAL_COMPONENTS']: 
        if optic in params.keys():
            for r in params[optic]:
                if 'l' in r.keys(): 
                    footprint = 'l: ' + repr(r['l']) + ', b:' + repr(r['b'])
                elif 'pointing' in r.keys():
                    footprint = 'pointing: ' + repr(r['pointing'])
                elif 'survey_footprint' in r.keys(): 
                    footprint = 'survey footprint: ' + r['survey_footprint'] 
                elif 'catalog' in r.keys():
                    footprint = 'catalog: ' + r['catalog']
                    
                print('  ' + optic + ' region=' + r['name'] 
                      + ' visit intervals=' + repr(r['visit_interval']) 
                      + ' footprint=' + footprint
                      + ' duration=' + repr(r['duration']) 
                      + ' nvisits=' + repr(r['nvisits'])
                     )

Number of time domain science cases = 13


Time domain authors and visit intervals [hrs] requested per region and filter:
Paladini2: 
  F213 region=Paladini_TDA visit intervals=[4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0] footprint=l: [25.0, 32.0], b:[2.0, 6.0] duration=3.5 nvisits=8
Benecchi: 
  F146 region=Benecchi1 visit intervals=[2920.0] footprint=pointing: [17.43229895, -14.63082839, 0.52] duration=730.0 nvisits=6
Kupfer: 
  F062 region=NGC6528 visit intervals=[0.01] footprint=pointing: [56.28615068, 22.57951507, 0.433] duration=8.0 nvisits=60
  F062 region=NGC6522 visit intervals=[0.01] footprint=pointing: [1.02461223, -3.92555662, 0.433] duration=8.0 nvisits=60
  F062 region=VVV_CL001 visit intervals=[0.01] footprint=pointing: [5.26747512, 0.77973258, 0.433] duration=8.0 nvisits=60
  F062 region=UKS1 visit intervals=[0.01] footprint=pointing: [268.6044167, -24.14661, 0.433] duration=8.0 nvisits=60
  F087 region=NGC6528 visit intervals=[0.01] footprint=pointing: [56.28615068, 22.5795

Note that Rich2 was unable to give specific cadence recommendations but suggested a 15min cadence (private communication). 

## Sky Regions for Time Domain Surveys

It's informative to examine what regions of sky have been requested for time-domain surveys. To make this easier the regions for each science case have been pre-calculated, so we can load them all here, then select out those for time domain science.

In [ ]:
science_regions = regions.load_regions_from_file(sim_config,
                                                         path.join(root_dir, 'config', 'rgps_time_domain_science_regions.json'))

In [ ]:
# Collect the regions for all time domain science cases.  Note not all of them may be present; this 
# happens if a science case is marked not ready for use.  This is done if insufficient information has been provided by 
# an author.
time_domain_regions = {} 
for author in authors_list:
    if author in science_regions.keys():
        time_domain_regions[author] = science_regions[author]

In the interests of identifying regions that may have been requested by different authors for different filters, we will combine the regions across all passbands, including spectroscopic. 

In [ ]:
# Compile a list of regions for this optic over all science cases 
region_list = []
region_names = []

for optic in sim_config['OPTICAL_COMPONENTS']: 

    for author, params in time_domain_regions.items():
        if optic in params.keys():

            # Do not duplicate a region if an author has requested it for multiple filters
            for r in params[optic]:
                if r.name not in region_names:
                    region_list.append(r)
                    region_names.append(r.name)

if len(region_list) > 0:
    r_merge = regions.combine_regions(region_list)
    r_merge.optic = 'ALL'
    r_merge.label = 'Combined survey footprint'

    mw1 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16, 10))
    mw1.title = r_merge.label + ' ' + r_merge.optic
    s = r_merge.pixels_to_skycoords()
    mw1.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=5, alpha=0.4)
    plt.rcParams.update({'font.size': 22})

    plt.tight_layout()
    plt.savefig(path.join(root_dir, 'time_domain_science', 'tda_all_requested_regions_map.png'))

print('Maximum HEALpixel value = ' + str(r_merge.region_map.max()))

## Priority regions for time domain science

We can now use this heatmap to identify regions of interest to multiple science cases from the HEALpixel values. 

To identify key regions of interest, we select HEALpixels requested by multiple science cases in multiple filters. 

In [ ]:
threshold = 4.0 # Minimum number of science cases requesting a HEALpixel in any filter

pixels = np.where(r_merge.region_map >= threshold)[0]

candidate_regions = {'pixel_set': pixels}

area = len(pixels) * PIXAREA
print('N overlap science cases=' + str(int(threshold)) 
      + ' N HEALpixels=' + str(len(pixels)) + ' area=' + str(round(area,2)) + 'sq.deg.')

# Plot these regions 
mw1 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16, 10))
mw1.title = 'Overlap of science regions'
proj = HEALPix(nside=64, order='ring', frame='icrs')

s = proj.healpix_to_skycoord(pixels)
mw1.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c='r', s=5, alpha=1.0)
plt.rcParams.update({'font.size': 22})

plt.tight_layout()
plt.savefig(path.join(root_dir, 'time_domain_science', 'all_tda_overlap_region_map_all_passbands.png'))


The candidate_regions entries per optic may have multiple clusters of HEALpixels, which need to be counted as separate regions.  

In [ ]:
def get_lrange(llist):

    idx1 = np.where(llist > 180.0)[0] 
    idx2 = np.where(llist <= 180.0)[0] 

    l0 = llist[idx1].min() 
    l1 = llist[idx2].max() 

    return l0, l1

In [ ]:
def cluster_pixels(params):
    
    # Convert the HEALpixels to a set of Skycoords in the galactic frame
    proj = HEALPix(nside=64, order='ring', frame='icrs')
    pixels = params['pixel_set']
    coords = proj.healpix_to_skycoord(pixels)
    coords = coords.transform_to('galactic')

    clusters = {}
    nc = -1
    max_sep = 2.5 * u.deg
    while len(coords) > 0:
        s = coords[0]
        
        sep = s.separation(coords)

        jdx = np.where(sep <= max_sep)[0]
        kdx = np.where(sep > max_sep)[0]
        
        l = coords[jdx].l.deg
        lmin = l.min() 
        lmax = l.max()
        print('RANGE: ',l, coords[jdx].b.deg)
        
        # Handle case if the region straddles the l-coordinate rollover
        if lmin > 0.0 and lmin < 180.0 and lmax > 180.0: 
            l0,l1 = get_lrange(l)
        else:
            l0 = l.min()
            l1 = l.max() 
        b0 = coords[jdx].b.deg.min()
        b1 = coords[jdx].b.deg.max()
        print('CORNERS: ', lmin, lmax, l0, l1, b0, b1)
        
        # Calculate the centroid as the mid-point of the diagonal between 
        # the opposing corners of a box encompasing the all HEALpixels in the region
        corner1 = SkyCoord(l0, b0, frame='galactic', unit=(u.deg, u.deg))
        corner2 = SkyCoord(l1, b1, frame='galactic', unit=(u.deg, u.deg))
        pa = corner1.position_angle(corner2)
        sep = corner1.separation(corner2)
        center = corner1.directional_offset_by(pa, sep/2)  
        
        cluster = { 
            'l_center': center.l.deg,
            'b_center': center.b.deg,
            'l0': l0,
            'l1': l1,
            'b0': b0,
            'b1': b1,
            'pixels': pixels[jdx]
        }
        nc += 1 
        
        clusters[nc] = cluster

        # Remove identified pixels from the coords list 
        coords = coords[kdx]
        pixels = pixels[kdx]
        
    params['clusters'] = clusters

    return params

In [ ]:
region_set = {}
params = cluster_pixels(candidate_regions)

# Filter out clusters that are unfeasibly large
for key, value in params.items():
    if key == 'clusters':
        subset = {}
        for cid,cluster in params['clusters'].items():
            #dl = abs(cluster['l0'] - cluster['l1'])
            #if len(cluster['pixels']) < 20.0 and dl < 20.0:
            print(' -> ', cid, cluster)
            subset[cid] = cluster
        params['clusters'] = subset
    else:
        print(key, value)
    
candidate_regions = params


In [ ]:
# Pixel clusters are separated by assuming they have to be a maximum angular separation apart
max_sep = 2.5*u.deg
optic_col = []
data = np.array([])
ids = []
centroids = np.array([])
for cid,cluster in candidate_regions['clusters'].items():
    s = SkyCoord(cluster['l_center'], cluster['b_center'], frame='galactic', unit=(u.deg, u.deg))

    # Check whether we already have a region at this location 
    if len(centroids) > 0: 
        cluster_centroids = SkyCoord(centroids[:,0], centroids[:,1], frame='galactic', unit=(u.deg, u.deg))
        sep = s.separation(cluster_centroids) 
        
        # If not, then add a new cluster
        if (sep >= max_sep).all():                
            ids.append(len(ids))
            data = np.vstack((data,[
                cluster['l_center'], cluster['b_center'], 
                cluster['l0'], cluster['l1'], cluster['b0'], cluster['b1']
            ]))

            optic_col.append(optic)

            centroids = np.vstack((centroids, [cluster['l_center'], cluster['b_center']]))
            
        # If the cluster is already known, compare the boundaries and extend if need be to  
        # form the cluster superset of pixels.  Also add the optic to the list of optics requesting 
        # this cluster
        else:
            #print('Existing clusters: ', cluster_centroids)
            #print('Candiddate cluster ',s)
            
            #print('Separations: ',sep, max_sep)
            
            cid = np.where(sep <= max_sep)[0][0]
            if len(data.shape) == 1: 
                cmatch = data
            else:
                cmatch = data[cid,:]
            superset = False 
            if superset:
                cmatch[2] = min(cmatch[2], cluster['l0']) 
                cmatch[3] = max(cmatch[3], cluster['l1']) 
                cmatch[4] = min(cmatch[4], cluster['b0']) 
                cmatch[5] = max(cmatch[5], cluster['b1'])
                cmatch[0] = np.median([cmatch[2],cmatch[3]])
                cmatch[1] = np.median([cmatch[4],cmatch[5]])
                if len(data.shape) == 1:
                    data = cmatch
                else:
                    data[cid,:] = cmatch
            if optic not in optic_col[cid]:
                optic_col[cid] = optic_col[cid] + ', ' + optic
            
    # If we have no clusters yet, simply add a new one 
    else:
        ids.append(len(ids))
        data = np.array([
            cluster['l_center'], cluster['b_center'], 
            cluster['l0'], cluster['l1'], cluster['b0'], cluster['b1']
        ])
        optic_col.append(optic)

        centroids = np.array([[cluster['l_center'], cluster['b_center']]])
                
data = np.array(data)

# Allow for testing with a single filter
if len(data.shape) == 1:
    TD_regions = Table([
        Column(name='ID', data=ids),
        Column(name='l_center', data=[data[0]]),
        Column(name='b_center', data=[data[1]]),
        Column(name='l0', data=[data[2]]),
        Column(name='l1', data=[data[3]]),
        Column(name='b0', data=[data[4]]),
        Column(name='b1', data=[data[5]])
    ])

else:
    TD_regions = Table([
        Column(name='ID', data=ids),
        Column(name='l_center', data=data[:,0]),
        Column(name='b_center', data=data[:,1]),
        Column(name='l0', data=data[:,2]),
        Column(name='l1', data=data[:,3]),
        Column(name='b0', data=data[:,4]),
        Column(name='b1', data=data[:,5])
    ])

TD_regions.pprint_all()

TD_regions.write(path.join(root_dir, 'time_domain_science', 'all_tda_candidate_tda_fields_table.txt'), format='ascii', overwrite=True)

Plot these selected regions for easy reference.

In [ ]:
def plot_outline(skymapplot, survey_region, ssmall=5.0, outline_color='red'):
    """
    Function to plot the outline of a survey footprint, given the survey region boundaries in the form of 
    survey_region = { 'l': [lmin, lmax], 'b': [bmin, bmax] }
    """

    l0 = survey_region['l'][0]
    l1 = survey_region['l'][1]
    if l0 > 0.0 and l0 > 180.0 and l1 < 180.0: 
        lrangeset = [np.arange(l0, 359.9, 0.1), np.arange(0.0, l1, 0.1)]
        brangeset = [
            np.arange(survey_region['b'][0], survey_region['b'][1], 0.1), 
            np.arange(survey_region['b'][0], survey_region['b'][1], 0.1), 
        ]
    else:
        lrangeset = [np.arange(l0, l1, 0.1)]
        brangeset = [np.arange(survey_region['b'][0], survey_region['b'][1], 0.1)]
    
    # Plot a boundary region if lrange or brange has non-zero length: 
    for xx in range(0, len(lrangeset), 1): 
        lrange = lrangeset[xx]
        brange = brangeset[xx]
        
        if len(lrange) > 0 and len(brange) > 0:
            # Since the plotting package supports only scatter plots, calculate a range of points 
            # to represent the outer boundaries.  
            llist = []
            blist = []
        
            # Right-hand edge of box
            llist += [survey_region['l'][0]]*len(brange)
            blist += brange.tolist()
        
            # Left-hand edge of box 
            llist += [survey_region['l'][1]]*len(brange)
            blist += brange.tolist()
        
            # Lower edge of box 
            llist += lrange.tolist()
            blist += [survey_region['b'][0]]*len(lrange)
        
            # Upper edge of box 
            llist += lrange.tolist()
            blist += [survey_region['b'][1]]*len(lrange)
    
            alpha = 0.4
            
        # Otherwise, this is a single-point region, so plot it as such
        else:
            llist = [survey_region['l'][0]]
            blist = [survey_region['b'][0]]
            alpha = 1.0
            
        # Add this outline to the plot; this has to be done in ICRS coordinates
        s = SkyCoord(llist, blist, frame='galactic', unit=(u.deg, u.deg))
        s = s.transform_to(ICRS)
        skymapplot.scatter(s.ra.deg*u.deg, s.dec.deg*u.deg, c=outline_color, s=ssmall, alpha=alpha)

    return skymapplot

In [ ]:
# Plot the selected TDA regions 
mw2 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16, 10))
mw2.title = 'Candidate time domain survey fields'
proj = HEALPix(nside=64, order='ring', frame='icrs')

for r in TD_regions:
    outline = {
        'l': [float(r['l0']), float(r['l1'])],
        'b': [float(r['b0']), float(r['b1'])]
    }
    #print(outline)
    plot_outline(mw2, outline)

plt.rcParams.update({'font.size': 22})

plt.tight_layout()
plt.savefig(path.join(root_dir, 'time_domain_science', 'all_tda_candidate_tda_fields.png'))


## Refining the Time Domain Regions 

A number of these science cases request very wide regions (e.g. most of the Galactic Plane), essentially indicating that anywhere within the footprint would be beneficial for time domain observations.  The analysis above shows that including these regions tends to result in a larger number of smaller regions where some large catalog entries (e.g. the AGN catalog) overlap with a very wide area footprint.  While this individual object is no doubt of interest, it causes the algorithm to simply re-create the footprint of the catalog, rather than select regions of interest to multiple science cases.  

A better approach is not to include these very-large area regions in the list of cases for building the regions themselves.  NOTE: THIS DOES NOT MEAN EXCLUDED REGIONS ARE IGNORED!  Instead we evaluate the the overlap for those science cases from the regions chosen later on when the metrics are calculated. 

In [ ]:
pop_list = ['DAmmando', 'Pascucci', 'Bahramian', 'Navarro']
time_domain_authors = []
for author in authors_list: 
    if author not in pop_list: 
        time_domain_authors.append(author)

time_domain_regions = {} 
for author in time_domain_authors:
    if author in science_regions.keys():
        time_domain_regions[author] = science_regions[author]

We can now repeat the above analysis with this revised list of science cases as a basis. 

In [ ]:
# Compile a list of regions for this optic over all science cases 
region_list = []
region_names = []

for optic in sim_config['OPTICAL_COMPONENTS']: 

    for author, params in time_domain_regions.items():
        if optic in params.keys():

            # Do not duplicate a region if an author has requested it for multiple filters
            for r in params[optic]:
                if r.name not in region_names:
                    region_list.append(r)
                    region_names.append(r.name)

if len(region_list) > 0:
    r_merge = regions.combine_regions(region_list)
    r_merge.optic = 'selected TDA science cases'
    r_merge.label = 'Combined survey footprint'

    mw1 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16, 10))
    mw1.title = r_merge.label + ' ' + r_merge.optic
    s = r_merge.pixels_to_skycoords()
    mw1.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=5, alpha=0.4)
    plt.rcParams.update({'font.size': 22})

    plt.tight_layout()
    plt.savefig(path.join(root_dir, 'time_domain_science', 'tda_selected_requested_regions_map.png'))

print('Maximum HEALpixel value = ' + str(r_merge.region_map.max()))

Select the regions prioritized by multiple science cases - note revised selection threshold since there are now fewer science cases per HEALpixel. 

In [ ]:
threshold = 3.0 # Minimum number of science cases requesting a HEALpixel in any filter

pixels = np.where(r_merge.region_map >= threshold)[0]

candidate_regions = {'pixel_set': pixels}

area = len(pixels) * PIXAREA
print('N overlap science cases=' + str(int(threshold)) 
      + ' N HEALpixels=' + str(len(pixels)) + ' area=' + str(round(area,2)) + 'sq.deg.')

# Plot these regions 
mw1 = MWSkyMap(projection='aitoff', grayscale=False, grid='galactic', background='infrared', figsize=(16, 10))
mw1.title = 'Overlap of science regions'
proj = HEALPix(nside=64, order='ring', frame='icrs')

s = proj.healpix_to_skycoord(pixels)
mw1.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c='r', s=5, alpha=1.0)
plt.rcParams.update({'font.size': 22})

plt.tight_layout()
plt.savefig(path.join(root_dir, 'time_domain_science', 'selected_tda_overlap_region_map_all_passbands.png'))


In [ ]:
region_set = {}
params = cluster_pixels(candidate_regions)

# Filter out clusters that are unfeasibly large
for key, value in params.items():
    if key == 'clusters':
        subset = {}
        for cid,cluster in params['clusters'].items():
            #dl = abs(cluster['l0'] - cluster['l1'])
            #if len(cluster['pixels']) < 20.0 and dl < 20.0:
            print(' -> ', cid, cluster)
            subset[cid] = cluster
        params['clusters'] = subset
    else:
        print(key, value)
    
candidate_regions = params

In [ ]:
# Pixel clusters are separated by assuming they have to be a maximum angular separation apart
max_sep = 1.0*u.deg
optic_col = []
data = np.array([])
ids = []
centroids = np.array([])
for cid,cluster in candidate_regions['clusters'].items():
    s = SkyCoord(cluster['l_center'], cluster['b_center'], frame='galactic', unit=(u.deg, u.deg))

    # Check whether we already have a region at this location 
    if len(centroids) > 0: 
        cluster_centroids = SkyCoord(centroids[:,0], centroids[:,1], frame='galactic', unit=(u.deg, u.deg))
        sep = s.separation(cluster_centroids) 
        
        # If not, then add a new cluster
        if (sep >= max_sep).all():                
            ids.append(len(ids))
            data = np.vstack((data,[
                cluster['l_center'], cluster['b_center'], 
                cluster['l0'], cluster['l1'], cluster['b0'], cluster['b1']
            ]))

            optic_col.append(optic)

            centroids = np.vstack((centroids, [cluster['l_center'], cluster['b_center']]))
            
        # If the cluster is already known, compare the boundaries and extend if need be to  
        # form the cluster superset of pixels.  Also add the optic to the list of optics requesting 
        # this cluster
        else:
            #print('Existing clusters: ', cluster_centroids)
            #print('Candiddate cluster ',s)
            
            #print('Separations: ',sep, max_sep)
            
            cid = np.where(sep <= max_sep)[0][0]
            if len(data.shape) == 1: 
                cmatch = data
            else:
                cmatch = data[cid,:]
            superset = False 
            if superset:
                cmatch[2] = min(cmatch[2], cluster['l0']) 
                cmatch[3] = max(cmatch[3], cluster['l1']) 
                cmatch[4] = min(cmatch[4], cluster['b0']) 
                cmatch[5] = max(cmatch[5], cluster['b1'])
                cmatch[0] = np.median([cmatch[2],cmatch[3]])
                cmatch[1] = np.median([cmatch[4],cmatch[5]])
                if len(data.shape) == 1:
                    data = cmatch
                else:
                    data[cid,:] = cmatch
            if optic not in optic_col[cid]:
                optic_col[cid] = optic_col[cid] + ', ' + optic
            
    # If we have no clusters yet, simply add a new one 
    else:
        ids.append(len(ids))
        data = np.array([
            cluster['l_center'], cluster['b_center'], 
            cluster['l0'], cluster['l1'], cluster['b0'], cluster['b1']
        ])
        optic_col.append(optic)

        centroids = np.array([[cluster['l_center'], cluster['b_center']]])
                
data = np.array(data)

# Allow for testing with a single filter
if len(data.shape) == 1:
    TD_regions = Table([
        Column(name='ID', data=ids),
        Column(name='l_center', data=[data[0]]),
        Column(name='b_center', data=[data[1]]),
        Column(name='l0', data=[data[2]]),
        Column(name='l1', data=[data[3]]),
        Column(name='b0', data=[data[4]]),
        Column(name='b1', data=[data[5]])
    ])

else:
    TD_regions = Table([
        Column(name='ID', data=ids),
        Column(name='l_center', data=data[:,0]),
        Column(name='b_center', data=data[:,1]),
        Column(name='l0', data=data[:,2]),
        Column(name='l1', data=data[:,3]),
        Column(name='b0', data=data[:,4]),
        Column(name='b1', data=data[:,5])
    ])

TD_regions.pprint_all()

TD_regions.write(path.join(root_dir, 'time_domain_science', 'selected_tda_candidate_tda_fields_table.txt'), format='ascii', overwrite=True)

In [ ]:
# Plot the selected TDA regions 
mw2 = MWSkyMap(grayscale=False, grid='galactic', background='infrared', figsize=(16, 10), 
               center=(0.0*u.deg, 0.0*u.deg), radius=(60.0*u.deg, 60.0*u.deg))

mw2.title = 'Candidate time domain survey fields'
proj = HEALPix(nside=64, order='ring', frame='icrs')

for r in TD_regions:
    outline = {
        'l': [float(r['l0']), float(r['l1'])],
        'b': [float(r['b0']), float(r['b1'])]
    }
    #print(outline)
    plot_outline(mw2, outline)

plt.rcParams.update({'font.size': 22})

plt.tight_layout()
plt.savefig(path.join(root_dir, 'time_domain_science', 'selected_tda_candidate_tda_fields.png'))

In [ ]:
# Close up of Galactic Center Field
l_cen = 0.0
b_cen = 0.0

r = TD_regions[5]

mw3 = MWSkyMap(grayscale=False, grid='galactic', background='infrared', figsize=(16, 10), 
               center=(l_cen*u.deg, b_cen*u.deg), radius=(10.0*u.deg, 10.0*u.deg))
mw3.title = 'Time domain field ' + str(r['ID']) 

# Plot shaded HEALpixel map of requested regions
s = r_merge.pixels_to_skycoords()
mw3.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=150, alpha=0.8)

# Plot outlines of time domain fields
outline = {
    'l': [float(r['l0']), float(r['l1'])],
    'b': [float(r['b0']), float(r['b1'])]
}
plot_outline(mw3, outline)

plt.rcParams.update({'font.size': 22})
plt.tight_layout()


In [ ]:
# Refined boundaries for W40/Serpens region
serpens = [
    {
        'l': [24.416209200921237, 31.8769619838468],
        'b': [1.4651381451102967, 3.6]
    },
    {
        'l': [26.9820729303582, 29.108464857799547],
        'b': [-0.4766073328462903, 1.4651381451102967]
    },
#    {
#        'l': [27.624388519213877, 28.47723108449249],
#        'b': [-0.4766073328462903, 0.42073947105053183]
#    }
]

In [ ]:
# W40
wforty = SkyCoord('18:31:29', '−02:05:24', frame='icrs', unit=(u.hourangle, u.deg)) 
wforty_gal = wforty.transform_to('galactic')
wforty_gal

In [ ]:
# Close up of W40/Serpens
l_cen = -30.0
b_cen = 0.0

mw3 = MWSkyMap(grayscale=False, grid='galactic', background='infrared', figsize=(16, 10), 
               center=(l_cen*u.deg, b_cen*u.deg), radius=(10.0*u.deg, 10.0*u.deg))
mw3.title = 'Time domain field: W40/Serpens'

# Plot shaded HEALpixel map of requested regions
s = r_merge.pixels_to_skycoords()
mw3.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=150, alpha=0.8)

# Plot outlines of time domain fields
for i in range(1,5,1):
    r = TD_regions[i]
    outline = {
        'l': [float(r['l0']), float(r['l1'])],
        'b': [float(r['b0']), float(r['b1'])]
    }
    plot_outline(mw3, outline)

# Plot refined boundary
for outline in serpens:
    plot_outline(mw3, outline, outline_color='yellow')

# Plot location of W40 
mw3.scatter(wforty.ra.deg * u.deg, wforty.dec.deg * u.deg, c='purple', s=100, alpha=0.8)

plt.rcParams.update({'font.size': 22})
plt.tight_layout()
plt.savefig(path.join(root_dir, 'time_domain_science', 'W40_Serpens_field_NIR.png'))

In [ ]:
# Close up of W40/Serpens
l_cen = -30.0
b_cen = 0.0

mw3 = MWSkyMap(grayscale=False, grid='galactic', background='optical', figsize=(16, 10), 
               center=(l_cen*u.deg, b_cen*u.deg), radius=(10.0*u.deg, 10.0*u.deg))
mw3.title = 'Time domain field: W40/Serpens'

# Plot shaded HEALpixel map of requested regions
s = r_merge.pixels_to_skycoords()
mw3.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=150, alpha=0.8)

# Plot outlines of time domain fields
for i in range(1,5,1):
    r = TD_regions[i]
    outline = {
        'l': [float(r['l0']), float(r['l1'])],
        'b': [float(r['b0']), float(r['b1'])]
    }
    plot_outline(mw3, outline)

# Plot refined boundary
for outline in serpens:
    plot_outline(mw3, outline, outline_color='yellow')

# Plot location of W40 
mw3.scatter(wforty.ra.deg * u.deg, wforty.dec.deg * u.deg, c='purple', s=100, alpha=0.8)

plt.rcParams.update({'font.size': 22})
plt.tight_layout()
plt.savefig(path.join(root_dir, 'time_domain_science', 'W40_Serpens_field_optical.png'))

In [ ]:
l_cen = -48.0
b_cen = 0.0

r = TD_regions[0]

mw3 = MWSkyMap(grayscale=False, grid='galactic', background='infrared', figsize=(16, 10), 
               center=(l_cen*u.deg, b_cen*u.deg), radius=(10.0*u.deg, 10.0*u.deg))
mw3.title = 'Time domain field ' + str(r['ID']) 

# Plot shaded HEALpixel map of requested regions
s = r_merge.pixels_to_skycoords()
mw3.scatter(s.ra.deg * u.deg, s.dec.deg * u.deg, c=r_merge.region_map[r_merge.pixels], cmap='Reds', s=150, alpha=0.8)

# Plot outlines of time domain fields
outline = {
    'l': [float(r['l0']), float(r['l1'])],
    'b': [float(r['b0']), float(r['b1'])]
}
plot_outline(mw3, outline)

plt.rcParams.update({'font.size': 22})
plt.tight_layout()


## Science from each Time Domain Region 

It is valuable now to check what science can be done within which TDA region.  This is most easily done by generating CelestialRegions for the selected fields, since these have HEALpixel maps which can be compared directly with those of the science cases.  

In [ ]:
tda_survey = {} 

for row in TD_regions:

    # Need to catch region boxes that extend over l=360/0deg and split them into two regions if they 
    # straddle the boundary
    if row['l0'] > 180.0 and row['l0'] > row['l1']:
        params = {
            "F213": [
                { # Nominal filter used to fit required structure
                "l": [row['l0'], 359.9],
                "b": [row['b0'], row['b1']],
                "nvisits": 1,
                "duration": 730.0,
                "visit_interval": [None],
                "name": str(row['ID'])
                },
                { 
                "l": [0.0, row['l1']],
                "b": [row['b0'], row['b1']],
                "nvisits": 1,
                "duration": 730.0,
                "visit_interval": [None],
                "name": str(row['ID'])
                },
            ],
            "comment": "None",
            "ready_for_use": "True"
        }

    else:
        params = {
            "F213": [{ # Nominal filter used to fit required structure
                "l": [row['l0'], row['l1']],
                "b": [row['b0'], row['b1']],
                "nvisits": 1,
                "duration": 730.0,
                "visit_interval": [None],
                "name": str(row['ID'])
                }],
            "comment": "None",
            "ready_for_use": "True"
        }
    tda_survey[str(row['ID'])] = params

tda_region_set = regions.build_region_maps(sim_config, tda_survey)

In [ ]:
science_per_tda_field = {key: {'cases': [], 'filters': [], 'visit_interval': []} for key in tda_region_set.keys()}

for author, science_params in time_domain_regions.items():

    # Make a list of all regions for this science case in all filters
    science_region_list = []
    for optic in sim_config['OPTICAL_COMPONENTS']:
        if len(science_params[optic]) > 0:
            science_region_list += science_params[optic]
    
    for survey_id,params in tda_region_set.items():
        overlap = False
        
        for r1 in params['F213']:
            for r2 in science_region_list:
                idx = set(r1.pixels).intersection(set(r2.pixels))
                if len(idx) > 0 and not overlap: 
                    overlap = True
                    science_per_tda_field[survey_id]['cases'].append(author)
                    science_per_tda_field[survey_id]['filters'] += [rr.optic for rr in science_region_list]
                    for rr in science_region_list:
                        if not np.isnan(rr.visit_interval[0]):
                            entries = [str(f) for f in rr.visit_interval.tolist()]
                            science_per_tda_field[survey_id]['visit_interval'] += entries

for field_id, params in science_per_tda_field.items():
    print('TDA region ' + str(field_id) + ' supports science case(s) ' + ','.join(params['cases']))
    print(' -> Filterset ' + ','.join(params['filters']))
    if len(params['visit_interval']) > 0:
        print(' -> Visit intervals ' + ','.join(params['visit_interval']))
    else:
        print(' -> No visit intervals defined')

Just as importantly, what science cases were not included in the selected regions?

In [ ]:
for author, science_params in time_domain_regions.items():
    in_region = False
    for field_id, params in science_per_tda_field.items():
        if author in params['cases']:
            in_region = True 

    if not in_region:
        print('Regions requested by ' + author + ' are not included in any region')